In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from stable_audio_tools.models.cvae import AudioCVAE

In [3]:
from typing import Sequence

In [4]:
model = AudioCVAE(cond_def=[5],
                  cond_embed_dim=128,
                  encoder_cfg={'in_channels': 2,
                               'channels': 128,
                               'c_mults': [1, 2, 4, 8, 16],
                               'strides': [2, 4, 4, 8, 8],
                               'latent_dim': 128,
                               'use_snake': True},
                  decoder_cfg={'out_channels': 2,
                               'channels': 128,
                               'c_mults': [1, 2, 4, 8, 16],
                               'strides': [2, 4, 4, 8, 8],
                               'latent_dim': 64,
                               'use_snake': True,
                               'final_tanh': False},
                  )

/home/sake/.local/share/virtualenvs/stable-audio-cvae-diFMr3O8/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [8]:
import torch
model.load_state_dict(torch.load("/home/sake/userdata/sake/stable-audio-cvae/ckpt/cvae_e0.pt")['model'])
model.eval()

/tmp/ipykernel_314107/4234767427.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/sake/userdata/sake/stable-audio-cvae/ckpt/cvae_e

AudioCVAE(
  (encoder): OobleckEncoder(
    (layers): Sequential(
      (0): Conv1d(2, 128, kernel_size=(7,), stride=(1,), padding=(3,))
      (1): EncoderBlock(
        (layers): Sequential(
          (0): ResidualUnit(
            (layers): Sequential(
              (0): SnakeBeta()
              (1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
              (2): SnakeBeta()
              (3): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
            )
          )
          (1): ResidualUnit(
            (layers): Sequential(
              (0): SnakeBeta()
              (1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
              (2): SnakeBeta()
              (3): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
            )
          )
          (2): ResidualUnit(
            (layers): Sequential(
              (0): SnakeBeta()
              (1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(27,), dilation=(9,))
    

In [12]:
B, C, T = 4, 2, 66150

In [14]:
import torch
wav = torch.randn(B, C, T)
labels = torch.randint(0, 5, (B, 1))  # e.g. {kick, hat, clap, ...}

In [17]:
import torchaudio
import torch.nn.functional as F
wav, sr = torchaudio.load("/home/sake/userdata/sake/oneshot_data/kick/001_50S1.wav")
wav = wav[:,:65536]
wav = F.pad(wav, (0, 65536 - wav.shape[1]), 'constant', 0)

In [18]:
labels = torch.tensor([0])
wav.shape, labels

(torch.Size([2, 65536]), tensor([0]))

In [19]:
rec, kl = model(wav.unsqueeze(0), labels.unsqueeze(0))

In [20]:
print("recon:", rec.shape, "kl:", float(kl))

recon: torch.Size([1, 2, 65536]) kl: 0.23954899609088898


In [24]:
from IPython.display import Audio
Audio(wav[0].numpy(), rate=44100)

In [25]:

Audio(rec[0].detach().numpy(), rate=44100)

: 